In [293]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pickle
from tqdm import trange

In [320]:
# we actually just need it to download cifar dataset
default_ds_train = torchvision.datasets.CIFAR10(train=True, download=True, root='../data/', transform=transforms.ToTensor())
default_ds_test = torchvision.datasets.CIFAR10(train=False, download=True, root='../data/', transform=transforms.ToTensor())

trainloader = DataLoader(default_ds_train, batch_size=32)
testloader = DataLoader(default_ds_test, batch_size=32)


Files already downloaded and verified
Files already downloaded and verified


In [340]:
class CiFaData(Dataset):
  def __init__(self, stage="train", transform=None, device="cpu"):
    self.device = device
    self.base_folder = "cifar-10-batches-py"
    self.transform = transform
    if stage == "train":
      batch_collection = [f"data_batch_{i}" for i in range(1, 5)]
    elif stage == "val":
      batch_collection = ["data_batch_5"]
    elif stage == "test":
      batch_collection = ["test_batch"]
    else:
      raise ValueError("Invalid stage, choose from train, val, test.")
    self.x_data = []
    self.y_data = []
    for batch in batch_collection:
      with open(f"../data/cifar-10-batches-py/{batch}", "rb") as f:
        data = pickle.load(f, encoding="latin1") 
        self.x_data.extend(data["data"])
        self.y_data.extend(data["labels"])
    self.y_data = torch.tensor(self.y_data)
    self.x_data = np.vstack(self.x_data).reshape(-1, 3, 32, 32) # from list to np stack; results in (N, 3, 32, 32)
    self.x_data = self.x_data.transpose((0, 2, 3, 1)) # into (N, H, W, C)
  def __len__(self):
    return len(self.y_data)
  def __getitem__(self, idx):
    if self.transform:
      return self.transform(self.x_data[idx]), self.y_data[idx]
    return transforms.ToTensor()(self.x_data[idx]).to(self.device), self.y_data[idx].to(self.device)

In [345]:
tf = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# tf = transforms.Normalize(0.5, 0.5)

device = 'cpu'

train_ds = CiFaData(stage="train", device=device)
val_ds = CiFaData(stage="val", device=device)
test_ds = CiFaData(stage="test", device=device)

In [352]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [353]:
class LittleConv(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,6,5) # out: (B, 6, 28, 28)
    self.pool = nn.MaxPool2d(2,2) # (B, 6, 14, 14)
    self.fc1 = nn.Linear(6 *14*14, 10) # (B, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = self.fc1(torch.flatten(x,1))
    return x

In [354]:
epochs = 2

littleconv = LittleConv()
optimimizer = optim.SGD(lr=0.001, params=littleconv.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()
littleconv.to(device)

for i in (t:= trange(epochs)):
  for x, y in train_loader:
    optimimizer.zero_grad()
    pred = littleconv(x)  
    loss = criterion(pred, y)
    loss.backward()
    optimimizer.step()

  t.set_description(f"epoch {i+1}: loss: {loss.item():.4f}")
f"that took {t.format_interval(t.format_dict['elapsed'])} minutes"

epoch 2: loss: 1.3500: 100%|██████████| 2/2 [00:13<00:00,  6.94s/it]


'that took 00:13'

In [325]:
# example net

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [ ]:
epochs = 20

# littleconv = LittleConv()
optimimizer = optim.SGD(lr=0.001, params=net.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()

for i in (t:= trange(epochs)):
  for x, y in train_loader:
    optimimizer.zero_grad()
    pred = net(x)  
    loss = criterion(pred, y)
    loss.backward()
    optimimizer.step()

  t.set_description(f"epoch {i+1}: loss: {loss.item():.4f}")